<a href="https://colab.research.google.com/github/kenwaldek/Whisper/blob/main/Blackscripted_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to use this script

this script turns file or url into transcripted text

multiple options to choose from: 1 if you have "video url" just run the code and it will ask to enter the link below. 2 upload file, click in the left sidebar on the folder icon and upload your file then when uploaded look bottom left blue circle needs to be gone & run the code

when script is finished it will spitt out different whispered files. click the file ... in file browser and download (if you dont see the files click the file refresh tab)

tip if you only want a small part of a video whispered you can use the screen reccording on your device and upload the file. for mac users it will be a .mov

files supported: .mp3, .mp4, .mpeg, .mpga, .m4a, .wav, .webm, .mov

# https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md

In [ ]:
# Install necessary packages
!pip install -q moviepy yt-dlp tqdm
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install -y ffmpeg

import os
import subprocess
import time
from moviepy.editor import VideoFileClip
from tqdm import tqdm
from threading import Thread, Event
import sys

# Define the directory path where the supported files are located
directory = '/content/'

# Supported file extensions
supported_extensions = ['.mp3', '.mp4', '.mpeg', '.mpga', '.m4a', '.wav', '.webm', '.mov']

# Get a list of all supported files in the directory
supported_files = [file for file in os.listdir(directory) if any(file.endswith(ext) for ext in supported_extensions)]

# Simple spinner function
def spinner_task(stop_event, description):
    spinner_chars = ['|', '/', '-', '\\']
    idx = 0
    print(f"{description}: ", end='', flush=True)
    while not stop_event.is_set():
        print(spinner_chars[idx % len(spinner_chars)], end='\r', flush=True)
        idx += 1
        time.sleep(0.1)
    print(f"{description}: Done!")

# Function to transcribe and translate audio to English with a spinner
def transcribe_to_english(file_path):
    command = ["whisper", file_path, "--model", "medium", "--task", "translate"]
    print(f"Transcribing {os.path.basename(file_path)}...")

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    stop_spinner = Event()
    spinner_thread = Thread(target=spinner_task, args=(stop_spinner, "Transcribing"))
    spinner_thread.start()

    for line in process.stdout:
        print(line.strip())

    stop_spinner.set()
    spinner_thread.join()

# Function to convert .mov to .mp3 with a spinner
def convert_mov_to_mp3(file_path):
    video = VideoFileClip(file_path)
    mp3_file = file_path.replace('.mov', '.mp3')
    print(f"Converting {os.path.basename(file_path)} to {os.path.basename(mp3_file)}...")

    stop_spinner = Event()
    spinner_thread = Thread(target=spinner_task, args=(stop_spinner, "Converting"))
    spinner_thread.start()

    video.audio.write_audiofile(mp3_file)

    stop_spinner.set()
    spinner_thread.join()
    print(f"Converted to {mp3_file}")
    return mp3_file

# Convert .mov files to .mp3 and transcribe
for file in supported_files:
    file_path = os.path.join(directory, file)

    if file.endswith('.mov'):
        mp3_file = convert_mov_to_mp3(file_path)
        transcribe_to_english(mp3_file)
    else:
        transcribe_to_english(file_path)

# Download from URL if no files are found
if len(supported_files) == 0:
    url = input("No supported files found. Please enter a URL to download an audio file: ")
    download_command = f"yt-dlp --extract-audio --audio-format mp3 -o '{directory}%(title)s.%(ext)s' {url}"
    print("Downloading audio...")

    stop_spinner = Event()
    spinner_thread = Thread(target=spinner_task, args=(stop_spinner, "Downloading"))
    spinner_thread.start()

    subprocess.call(download_command, shell=True)

    stop_spinner.set()
    spinner_thread.join()

    downloaded_files = [file for file in os.listdir(directory) if file.endswith('.mp3')]
    if downloaded_files:
        mp3_file = downloaded_files[0]
        print(f"Downloaded audio: {mp3_file}")
        transcribe_to_english(os.path.join(directory, mp3_file))
    else:
        print("No .mp3 files found after download.")
